# Simon's algorithm


**Input:** some oracle function $f:\{0, 1\}^n \rightarrow \{0, 1\}^n$ and such $f$ is $s$ periodic
$f(x) = f(y)$ iff $x \oplus y = 0$ or $x \oplus y = s$

**Goal:** find $s$


This is a Hidden Subgroup Problem with: $G = \{0, 1\}^n$, $G\geq H = \{0^n, s\}$

As other HSP, we solve it using a Quantum Fourier Transform:

**Step 1** allocate two registers $A$ and $B$

**Step 2** generate a uniform superposition on $A$ using a Hadamard transform (which coincides with G's Fourier Transform)

**Step 3** apply the oracle on $A$ and $B$

**Step 4** apply another Hadamard transform on $A$ 

**Step 5** measure $A$


The final sampling will produce bit-strings in $H^\bot = \{y | y\cdot s = 0\}$

Remains to gather $O(n)$ samples and find $s$ by inverting a linear system.


## Oracle generation

To keep the oracle model simple, we will restrict ourselves to linear operators over $\mathbb{F}_2^n$.

Since $f$ is required to be 2-to-1, I propose the following random generation procedure:

1) pick a random invertible linear operator $A$ over $\mathbb{F}_2^n$

2) define $f$ as $x \mapsto ((Ax)_1, ..., (Ax)_{n-1})$

This function is :
-  $A^{-1}e_{n}$ periodic
-  2-to-1.
-  very simple to implement using CNOT gates


**Note:** If you have a better idea, go ahead :D I picked this because linear operators are really simple to implement using CNOT gates and I'm lazy! You can also throw in some random bit flips.


In [ ]:
import numpy as np
from qat.lang.AQASM import *
from qat.qpus import get_default_qpu

def random_oracle(nbbits):
    A = np.eye(nbbits, dtype=np.byte)
    # Generating the oracle (simple since A is linear)
    routine = QRoutine()
    ins = routine.new_wires(nbbits)
    outs = routine.new_wires(nbbits - 1)
    with routine.compute():
        for _ in range(nbbits ** 2):
            i, j = np.random.choice(list(range(nbbits)), size=2, replace=False)
            A[j] ^= A[i]
            CNOT(ins[i], ins[j])
    for qb1, qb2 in zip(ins, outs):
        CNOT(qb1, qb2)
    routine.uncompute()
    return routine, A

oracle, secret = random_oracle(5)
%qatdisplay oracle --svg

In [ ]:
def rowechelon(A, B=None):
    """ Row echelon form """
    for i in range(A.shape[1]):
        pivot = None
        for j in range(i, A.shape[0]):
            if A[j][i]:
                pivot = j
                break
        if pivot is None:
            continue
        if pivot != i:
            A[[i, pivot]] = A[[pivot, i]]
            if B is not None:
                B[[i, pivot]] = B[[pivot, i]]
    
        for j in range(i+1, A.shape[0]):
            if A[j][i]:
                A[j] ^= A[i]
                if B is not None:
                    B[j] ^= B[i]
    
    
def nullspace(A):
    """ Computes a non-trivial generator of the nullspace of A """
    rowechelon(A)
    B = np.eye(A.shape[1], dtype=np.byte)
    A = A.T
    rowechelon(A, B)
    for r1, r2 in zip(A, B):
        if all(b == 0 for b in r1):
            return r2

def quantum_solve(oracle, nbbits):
    
    prog = Program()
    reg1 = prog.qalloc(nbbits)
    reg2 = prog.qalloc(nbbits - 1)

    for qbit in reg1:
        H(qbit)
    oracle(reg1, reg2)
    for qbit in reg1:
        H(qbit)
    job = prog.to_circ().to_job(nbshots=3 * nbbits, qubits=[reg1])
    qpu = get_default_qpu()
    result = qpu.submit(job)
    system = np.array([list(map(int, sample.state.value[0])) for sample in result])
    
    # All rows in `system` are orthogonal to `s`
    # Computing an element in the nullspace of `system`
    attempt = nullspace(system)
    return attempt

s = quantum_solve(oracle, 5)
print(s)
print(secret.dot(s) % 2)